In [0]:
import sys,ast,os,time
from cashflow_aua_aum import *
from log import syslog,sysError
from current_aua_aum import *
from file_reader import *
from staging_files import *
from distribution_volumes import *
from spark_conf import sparkConfig
from transformations import *
from baseline_data import *
from staging_file_2_fbi_work import *
from existingclientvetting import *
from trc_export_final import *
from copy_of_fsecpt_rollup import *
from CET_Rollup import *
from Adjustable_Data import *
import pyarrow as pa
import pyarrow.parquet as pq
from s3_connection import *

param = ast.literal_eval(sys.argv[1])
# create spark session
spark = sparkConfig("FSECPT")

def main():
    # try exception block to handle exceptions
    try:
        vcats_bucket = param['VCATS_S3_BUCKET']
        crm_bucket = param['CRM_S3_BUCKET']
        target_dir = param['TGT_S3_BUCKET']
        ftp_bucket = param['FTP_S3_BUCKET']
        target_path = param['DNI_S3_BUCKET']
        conn = est_connection(target_path)

        #create tables for input excel files
        read_files(spark, ftp_bucket, target_dir + "input_files/")
        #Methods for Cashflow AUA and AUM
        df_plan_fnd_dim_filtered = iig_plnfnd_dim_import_job(crm_bucket,spark)
        df_plan_dim = vcats_plan_dim_import_job(vcats_bucket,spark)
        #workbook - CET_Cashflow_AUA, Sheet - Joined3
        aua_df = cashflow_aua(vcats_bucket, df_plan_fnd_dim_filtered,spark,target_dir)
        write_output_to_parquet(aua_df, target_dir,"cashflow_aua", "overwrite")

        #workbook - CET_Cashflow_AUM, Sheet - Joined3
        aum_df = cashflow_aum(vcats_bucket, df_plan_fnd_dim_filtered, spark,target_dir)
        write_output_to_parquet(aum_df, target_dir,"cashflow_aum", "overwrite")

        #workbook - CET_Current_VCATS, SheetS - Current_AUM, Current_AUA,DC_NQP_counts,Current_Plans,Current_Plans_v2
        df_DC_NQP_Plan_count,df_current_aum,df_current_aua,df_current_clnt_plan,df_current_plans = cet_current_vcats(spark,df_plan_fnd_dim_filtered,df_plan_dim, vcats_bucket, target_dir)
        write_output_to_parquet(df_current_plans, target_dir, "current_plans", "overwrite")
        write_output_to_parquet(df_current_clnt_plan, target_dir, "current_plans_v2", "overwrite")
        write_output_to_parquet(df_DC_NQP_Plan_count, target_dir, "dc_nqp_counts", "overwrite")
        write_output_to_parquet(df_current_aum, target_dir, "Current_AUM", "overwrite")
        write_output_to_parquet(df_current_aua, target_dir, "Current_AUA", "overwrite")

        #workbook - DistributionVolumes_Staging_file, Sheet - Distribution_Volumes_2019
        df_distribution_volumes = distribution_volumes(spark, df_plan_fnd_dim_filtered)
        write_output_to_parquet(df_distribution_volumes, target_dir, "distribution_volumes", "overwrite")
        #workbook - Expense_calc, Sheet - Ongoing_Expense_and_Margin
        df_FSP_Revenue_Components = get_expense_calc(spark)
        write_output_to_parquet(df_FSP_Revenue_Components, target_dir, "ongoing_exp_margin", "overwrite")
        # workbook - Historic_Participant_Growth, Sheet - AverageGrowth, no data in test
        #df_AvgGrowth = read_parquet(target_dir + "hist_part_growth", spark) # use it for testing in SAT
        df_AvgGrowth = get_hist_part_growth(spark)
        write_output_to_parquet(df_AvgGrowth, target_dir, "hist_part_growth", "overwrite")
        # workbook - Staging File 0 Bill Code 447, Sheet - Fee_Disclosure
        df_fee_disclosure = get_fee_disclosure(spark)
        write_output_to_parquet(df_fee_disclosure, target_dir, "stg_0_fee_disclosure", "overwrite")
        # workbook - Staging File 2_2 Ancil CATS for Baseline, Sheet - FSP_Ancil_cats
        df_fsp_ancil_Cats = get_ancil_rev_Cats(spark)
        write_output_to_parquet(df_fsp_ancil_Cats, target_dir, "stg_2_2_ancil_cats", "overwrite")
        #workbook- staging File 1
        df_add_vision_id,df_Add_Plan_Part_Count = add_plan_part_count(spark)
        write_output_to_parquet(df_add_vision_id, target_dir, "stg_1_add_vision_id", "overwrite")
        write_output_to_parquet(df_Add_Plan_Part_Count, target_dir, "stg_1_add_plan_part_count", "overwrite")
        # workbook- baseline_data sheet - AUMandAUMadded,baseline_data,baseline_w_Part_Growth,baseline_w_Part_Growth_v2
        df_curr_AUA_AUM,df_baseline_data,df_baseline_w_Part_Growth,df_baseline_w_Part_Growth_v2 = get_baseline_data(spark, df_AvgGrowth, df_fsp_ancil_Cats, target_dir)
        write_output_to_parquet(df_curr_AUA_AUM, target_dir, "AUMandAUMadded", "overwrite")
        write_output_to_parquet(df_baseline_data, target_dir, "baseline_data", "overwrite")
        write_output_to_parquet(df_baseline_w_Part_Growth, target_dir, "baseline_w_Part_Growth", "overwrite")
        write_output_to_parquet(df_baseline_w_Part_Growth_v2, target_dir, "baseline_w_Part_Growth_v2", "overwrite")
        #workbook - staging file 2 fbi work continued
        trc_work,seg_avg_edit = get_trc_work(spark,df_Add_Plan_Part_Count)
        write_output_to_parquet(trc_work, target_dir, "stg_2_trc_work", "overwrite")
        write_output_to_parquet(seg_avg_edit, target_dir, "stg_2_seg_avg_edit", "overwrite")
        #workbook - compliance testing, sheet - Compliance_Testing_Volume
        df_compliance_testing_volume = get_compliance_testing_volume(spark)
        write_output_to_parquet(df_compliance_testing_volume, target_dir, "compliance_testing_volume", "overwrite")
        # workbook - Staging File 2_1 Ancil Rev Categories, sheet - Ancil_Cats, Baseline_w_AncilRevCat
        df_ancil_cats,df_baseline_ancil = get_ancil_cats(spark,df_baseline_data)
        write_output_to_parquet(df_ancil_cats, target_dir, "Ancil_Cats", "overwrite")
        write_output_to_parquet(df_baseline_ancil, target_dir, "baseline_w_ancil", "overwrite")
        # workbook-staging file 8 trc work
        df_percentile_rev_avg_bal, df_act_exp_add_seg, df_exp_driver = get_staging_file_8_trc_work(spark)
        write_output_to_parquet(df_percentile_rev_avg_bal, target_dir, "percentile_rev_avg_bal", "overwrite")
        write_output_to_parquet(df_act_exp_add_seg, target_dir, "act_exp_add_seg", "overwrite")
        write_output_to_parquet(df_exp_driver, target_dir, "exp_driver", "overwrite")
        # workbook - Baseline expense
        df_total_expense, df_sheet22 = get_baseline_expense(spark)
        write_output_to_parquet(df_total_expense, target_dir, "baseline_expense_total_expense", "overwrite")
        write_output_to_parquet(df_sheet22, target_dir, "baseline_expense_sheet22", "overwrite")
        # workbook - existing client vetting, sheet - cat vetting score,per_part_expense
        df_cat_vetting_score = get_cat_vetting_score(spark, df_curr_AUA_AUM)
        write_output_to_parquet(df_cat_vetting_score, target_dir, "cat_vetting_score", "overwrite")
        df_per_part_exp = get_per_part_exp(spark)
        write_output_to_parquet(df_per_part_exp, target_dir, "per_part_expense", "overwrite")
        #workbook - staging file 3
        df_client_rollup, df_add_baseline_data = get_add_baseline_data(spark,trc_work,df_baseline_ancil)
        write_output_to_parquet(df_client_rollup, target_dir, "staging_file_3_client_rollup", "overwrite")
        write_output_to_parquet(df_add_baseline_data, target_dir, "add_baseline_data", "overwrite")

        # workbook - staging file 4 start of ancil recommendation
        add_ancil_rev, asc_pricing = get_asc_pricing(spark, seg_avg_edit, df_add_baseline_data, df_ancil_cats)
        write_output_to_parquet(add_ancil_rev, target_dir, "staging_file_4_add_ancil_rev", "overwrite")
        write_output_to_parquet(asc_pricing, target_dir, "staging_file_4_asc_pricing", "overwrite")

        # workbook - staging file 5
        add_bill_volume_edit = get_add_bill_volume(spark, asc_pricing)
        write_output_to_parquet(add_bill_volume_edit, target_dir, "staging_file_5_add_bill_volume_edit", "overwrite")

        # workbook -staging file 6
        DistributionEditV2, add_avg_billed = get_add_avg_billed(spark, add_bill_volume_edit, df_distribution_volumes, df_fee_disclosure, df_compliance_testing_volume)
        write_output_to_parquet(DistributionEditV2, target_dir, "staging_file_6_DistributionEditV2", "overwrite")
        write_output_to_parquet(add_avg_billed, target_dir, "staging_file_6_add_avg_billed", "overwrite")

        #workbook - Staging File 7 Ancillary work, sheet - Ancil_work,Ancillary_Fee_Export. It's ideal to derive second data frame in dependent workbook
        #df_Add_Avg_Billed = read_csv(ftp_bucket,"test_input/Add_Avg_Billed.csv",",",spark)
        df_Ancil_work = get_stg_7_ancillary_work(spark,add_avg_billed)
        write_output_to_parquet(df_Ancil_work, target_dir, "Ancil_work", "overwrite")

        #workbook - staging file 9
        blcl_df = read_csv(ftp_bucket,"test_input/BLCL.csv",",",spark)
        # client_sens_rpt_df = read_csv(ftp_bucket,"test_input/Client_Sensitivity_Rpt.csv",",",spark)
        client_sens_rpt_df = spark.sql(
            """with clnt_rank as (SELECT 
                van_companyid,van_ratingdate,van_ratingid,van_relationshipcategoryid_value,van_ratingtype,
                RANK() OVER (partition by van_companyid ORDER BY van_ratingdate DESC) AS Rank
                from inst_dna_compf_crm.relationshipratings
                where van_relationshipcategoryid_value = 'Sensitivity / Rel. Status' and van_ratingtype = 'Full Service')
            select distinct van_companyid,van_ratingdate,van_ratingid,van_relationshipcategoryid_value,van_ratingtype,Rank from clnt_rank where rank = 1""")
        df_add_tenure = get_add_tenure(spark,df_act_exp_add_seg,blcl_df,add_ancil_rev,df_percentile_rev_avg_bal,client_sens_rpt_df)
        write_output_to_parquet(df_add_tenure, target_dir, "staging_file_9_add_tenure", "overwrite")

        # workbook - Staging File 10 REC work cont, sheet - Add_Non_Act_uplift
        #df_add_tenure = read_csv(ftp_bucket,"test_input/Add_Tenure.csv",",",spark)
        df_Non_Act_uplift = get_Non_Act_uplift(spark, df_Ancil_work, df_add_tenure, df_baseline_w_Part_Growth,ftp_bucket)
        write_output_to_parquet(df_Non_Act_uplift, target_dir, "Non_Act_uplift", "overwrite")

        # workbook - Staging File 11 REC work cont, sheet - TRC_Export_final
        df_target_margins = read_csv(ftp_bucket, "test_input/FSP_Target_Margins.csv", ",", spark)
        df_get_TRC_Export_final,df_get_TRC_Export = get_TRC_Export_final(spark, df_add_vision_id, df_Non_Act_uplift, df_target_margins,df_exp_driver)
        write_output_to_parquet(df_get_TRC_Export_final, target_dir, "TRC_Export_final", "overwrite")
        write_output_to_parquet(df_get_TRC_Export, target_dir, "TRC_Export", "overwrite")

        #workbook -staging file 12
        # df_get_TRC_Export_final = read_csv(ftp_bucket,"test_input/TRC_Export_Final_Copy.csv")
        TRC_Export_V2,ancil_n_dist,final_ancillary_export = get_trc_export_v2(spark, df_get_TRC_Export_final, DistributionEditV2, df_Ancil_work,df_current_plans)
        write_output_to_parquet(TRC_Export_V2, target_dir, "staging_file_12_TRC_Export_V2", "overwrite")
        write_output_to_parquet(ancil_n_dist, target_dir, "staging_file_12_ancil_n_dist", "overwrite")
        write_output_to_parquet(final_ancillary_export, target_dir, "staging_file_12_final_ancillary_export", "overwrite")

        #workbook- staging file 13
        # ancil_n_dist = read_csv(ftp_bucket,"test_input/Ancil_n_Dist_Copy.csv",",",spark)
        AncilCats = get_AncilCats(spark,df_ancil_cats,ancil_n_dist)
        write_output_to_parquet(AncilCats, target_dir, "staging_file_13_AncilCats", "overwrite")

        # workbook-CET Rollup
        #Final_Group_CF_AUA = read_csv(ftp_bucket, "test_input/Final_Group_CF_AUA_copy.csv", ",", spark)
        Tenure = read_csv(ftp_bucket, "test_input/Tenure_copy.csv", ",", spark)
        # df_cat_vetting_score = read_csv(ftp_bucket, "test_input/cat_vetting_score_copy.csv", ",", spark) # use it for testing in SAT
        Rollup = get_Rollup(spark, df_client_rollup, df_baseline_w_Part_Growth, aum_df, df_AvgGrowth,aua_df, Tenure, df_FSP_Revenue_Components,ancil_n_dist, df_get_TRC_Export, df_cat_vetting_score, df_DC_NQP_Plan_count, df_total_expense,df_current_clnt_plan)
        write_output_to_parquet(Rollup, target_dir, "CET_Rollup", "overwrite")

        #workbook- copy of FSECPT Sheet # Sheet5
        BLCL = read_csv(ftp_bucket,"test_input/BLCL.csv",",",spark)
        # Roll_Up = read_csv(ftp_bucket,"test_input/RollUp_copy.csv",",",spark)
        Joined4 = get_copy_fsecpt_rollup(spark,df_per_part_exp,BLCL,Rollup,AncilCats,client_sens_rpt_df)
        write_output_to_parquet(Joined4, target_path, "bsln_adj_recc","overwrite")

        #workbook - Adjustable data
        sheet4 = read_csv(ftp_bucket,"test_input/Adjustable_data_sheet4_copy.csv",",",spark)
        Ancil_Fee,sheet1 = get_adjustable_data(spark,sheet4,final_ancillary_export,df_baseline_w_Part_Growth_v2,df_get_TRC_Export_final,df_sheet22,AncilCats)
        clnt_fee_df = Ancil_Fee.toPandas()
        clnt_fee_table = pa.Table.from_pandas(clnt_fee_df, preserve_index=False)
        clnt_demo_df = sheet1.toPandas()
        clnt_demo_table = pa.Table.from_pandas(clnt_demo_df, preserve_index=False)
        pq.write_table(clnt_demo_table, target_path + "fcpt_clnt_demo/clnt_demo.parquet",flavor='spark', filesystem=conn)
        pq.write_table(clnt_fee_table, target_path + "fcpt_clnt_fee/clnt_fee.parquet",flavor='spark', filesystem=conn)
        # write_output_to_parquet(sheet1, target_path, "fcpt_clnt_demo", "overwrite")
        # write_output_to_parquet(Ancil_Fee, target_path, "fcpt_clnt_fee", "overwrite")
        # spark.stop()
        # time.sleep(60)
        # demo_target_path = target_path+"fcpt_clnt_demo"
        # os.system("hadoop distcp {}/part*  {}.parquet".format(demo_target_path, demo_target_path + "/clnt_demo"))
        # os.system("hadoop fs -rm -r {}/part*".format(demo_target_path))
        #
        # ancil_target_path = target_path + "fcpt_clnt_fee"
        # os.system("hadoop distcp {}/part*  {}.parquet".format(ancil_target_path, ancil_target_path + "/clnt_fee"))
        # os.system("hadoop fs -rm -r {}/part*".format(ancil_target_path))

    except Exception as e:
        sysError('Method :\'{}\' failed with exception : '.format(main.__name__), e)

if __name__ == '__main__':
    main()